In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
# from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score#, train_test_split

%matplotlib inline
plt.style.use('fivethirtyeight')

df_train = pd.read_csv('../data/iowa_housing/train.csv', usecols = [1,17, 18, 19, 43,44,46,49,50,70,77,80])
df_test = pd.read_csv('../data/iowa_housing/test.csv', usecols = [1,17, 18, 19, 43,44,46,49,50,70,77])

In [3]:
df_train.head()

,MSSubClass,OverallQual,OverallCond,YearBuilt,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,ScreenPorch,YrSold,SalePrice
0,60,7,5,2003,856,854,1710,2,1,0,2008,208500
1,20,6,8,1976,1262,0,1262,2,0,0,2007,181500
2,60,7,5,2001,920,866,1786,2,1,0,2008,223500
3,70,7,5,1915,961,756,1717,1,0,0,2006,140000
4,60,8,5,2000,1145,1053,2198,2,1,0,2008,250000


In [4]:
df_test.head()

,MSSubClass,OverallQual,OverallCond,YearBuilt,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,ScreenPorch,YrSold
0,20,5,6,1961,896,0,896,1,0,120,2010
1,20,6,6,1958,1329,0,1329,1,1,0,2010
2,60,5,5,1997,928,701,1629,2,1,0,2010
3,60,6,6,1998,926,678,1604,2,1,0,2010
4,120,8,5,1992,1280,0,1280,2,0,144,2010


In [12]:
X_train = df_train.loc[:, df_train.columns != 'SalePrice']
y_train = np.log(df_train['SalePrice'])

In [18]:
def rmse(estimator, X, y):
    preds = estimator.predict(X)
    error = np.sqrt(np.mean((preds - y)**2))
    return error

In [6]:
rf = RandomForestRegressor()

In [7]:
rf.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [17]:
rf_scores = []
spl = [1, 2, 4, 8]
mx_ft = [1, 0.5, 'sqrt', 'log2']

for m in mx_ft:
    for s in spl:
        rf.set_params(max_features = m, min_samples_leaf = s, n_jobs = -1)
        scores = cross_val_score(estimator = rf, X = X_train, y = y_train, scoring = rmse, cv = 10)
        rf_scores.append((np.mean(scores), m, s))

In [19]:
min(rf_scores)

(0.15827625323117767, 'log2', 1)

In [20]:
rf.set_params(max_features = 'log2', min_samples_leaf = 1, n_jobs = -1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [21]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [22]:
df_test['prediction'] = rf.predict(df_test)

In [35]:
df_submission = pd.read_csv('../data/iowa_housing/sample_predictions.csv')

In [36]:
df_submission['SalePrice'] = np.exp(df_test['prediction'])

In [38]:
df_submission.to_csv('../data/iowa_housing/ts_predictions.csv', index=False)